<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/Batch_Invoices_to_Singles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import PyPDF4
from io import BytesIO
import csv

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'
csv_filename = os.path.join(pdf_upload, 'Batch.csv')


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Keep track of processed pages for each invoice
processed_invoices = set()
csv_file = 'Batch.csv'
if os.path.isfile(csv_file):
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            processed_invoices.add(row[0])

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 1
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Extract the invoice number and date from the PDF file
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF and extract the invoice number and date
            for i in range(num_pages):
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
                match_page = re.search(r'PAGE:\s*1', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # If the invoice has already been processed, skip it
                    if invoice_id in processed_invoices:
                        print(f"Invoice {invoice_id} has already been processed, skipping")
                        continue
                    
                    # Define the output filename
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # If the output file already exists, add the current page to it
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                num_existing_pages = existing_pdf.getNumPages()
                                
                                # Create a new PDF writer object and add the existing pages to it
                                pdf_writer = PyPDF4.PdfFileWriter()
                                for page_num in range(num_existing_pages):
                                    pdf_writer.addPage(existing_pdf.getPage(page_num))

                                # Add the current page to the PDF writer object
                                pdf_writer.addPage(page)

                                # Write the updated PDF file to a BytesIO object
                                buffer = BytesIO()
                                pdf_writer.write(buffer)

                                # Write the updated PDF file to disk
                                with open(invoice_filename, 'wb') as f_out:
                                    f_out.write(buffer.getvalue())
                                    x = x + 1
                                    print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                            except Exception as e:
                                print(f"Failed to add existing pages to {invoice_filename}: {e}")
                    else:
                        # Create a new PDF writer object and add the current page to it
                        pdf_writer = PyPDF4.PdfFileWriter()
                        pdf_writer.addPage(page)

                        # Write the new PDF file to disk
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 1
                            print(f"Used {filename} page {i+1} to {invoice_id} page {x}")

            # Add the invoice to the set of processed invoices
            processed_invoices.add(invoice_id)
            # Write the updated set of processed invoices to the CSV file
#                with open(csv_filename, 'a', newline='') as f:
#                    writer = csv.writer(f)
#                    writer.writerow([invoice_id])

In [ ]:
#!!! This Works!!!

import os
import re
import PyPDF4
from io import BytesIO

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Keep track of processed pages for each invoice
processed_invoices = set()

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 1
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Extract the invoice number and date from the PDF file
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF and extract the invoice number and date
            for i in range(num_pages):
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
                match_page = re.search(r'PAGE:\s*1', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # Define the output filename
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # If the output file already exists, add the current page to it
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                for page_num in range(existing_pdf.getNumPages()):
                                    pdf_writer.insertPage(existing_pdf.getPage(page_num), 0)
                                    x = x + 1
                                    print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                            except Exception as e:
                                print(f"Failed to add existing pages to {invoice_filename}: {e}")
                            
                            # Append the current page to the existing PDF file
                            with open(invoice_filename, 'ab') as f_out:
                                pdf_writer.write(f_out)
                                x = x + 1
                                print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                    else:
                        # Create a new PDF writer object and add the current page to it
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 1
                            print(f"Used {filename} page {i+1} to {invoice_id} page {x}")

                       
                        processed_invoices.add(invoice_id)

                        

In [ ]:
!!!!! WORKING BACKUP!!!!

import os
import glob
import re
import PyPDF4
import csv
import camelot
import pandas as pd
from pdfminer.high_level import extract_text
from google.colab import files
import openpyxl
from google.colab import drive
from io import BytesIO

drive.mount('/content/drive')

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 0
    current_invoice_id = None
    current_output_pdf = None
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Open the PDF file in read-binary mode
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF
            for i in range(num_pages):
                # Get the current page and extract the text
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # If current_invoice_id is None or does not match invoice_id, close the current output PDF and create a new one
                    if current_invoice_id is None or current_invoice_id != invoice_id:
                        if current_output_pdf is not None:
                            current_output_pdf.close()
                        invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                        current_output_pdf = open(invoice_filename, 'wb')
                        current_invoice_id = invoice_id
                        x = 1
                        print(f"Used {filename} page {i+1} to {invoice_id} page {x}")
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # Write the contents of the BytesIO object to a new PDF file
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                for page_num in range(existing_pdf.getNumPages()):
                                    pdf_writer.insertPage(existing_pdf.getPage(page_num), 0)
                            except Exception as e:
                                #print(f"Failed to add existing pages to {invoice_id}: {e}")
                                continue  # continue the loop execution without appending the current page
                                                
                            # Append the current page to the existing PDF file
                            with open(invoice_filename, 'ab') as f_out:
                                pdf_writer.write(f_out)
                                x = x + 1
                                print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                    else:
                        # If the file doesn't exist, create it and write the current page to it
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 0
                            print(f"Used {filename} page {i+1} to {invoice_id}")

